In [ ]:
import numpy as np
import h5py
import labrotation.file_handling as fh
from matplotlib import pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import matplotlib.colors as mcolors
from datetime import datetime
import json
from labrotation import json_util
import datadoc_util
import pandas as pd
import seaborn as sns
import os

In [ ]:
def get_datetime_for_fname():
    now = datetime.now()
    return f"{now.year:04d}{now.month:02d}{now.day:02d}-{now.hour:02d}{now.minute:02d}{now.second:02d}"

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    data_docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    data_docu_folder = fh.open_dir("Open Data Documentation folder")
print(data_docu_folder)

In [ ]:
ddoc = datadoc_util.DataDocumentation(data_docu_folder)
ddoc.loadDataDoc()

In [ ]:
output_folder = env_dict["DOWNLOADS_FOLDER"]
print(f"Output files will be saved to {output_folder}")

In [ ]:
whole_traces_h5_fpath = fh.open_file("Open traces h5 file!")
print(whole_traces_h5_fpath)

# Automatize for whole dataset

In [ ]:
dset_folder = fh.open_dir("Select folder that contains all data to be considered (traces.h5 files)")

In [ ]:
dict_uuid_traces = dict()
for root,dirs,files in os.walk(dset_folder):
        for file in files:
            if "_traces" in file and os.path.splitext(file)[-1] == ".h5":
                fpath = os.path.normpath(os.path.join(root, file))
                with h5py.File(fpath, "r") as hf:
                    uuid = hf.attrs["uuid"]
                    traces = hf["traces"][()]
                    assert uuid not in dict_uuid_traces
                    dict_uuid_traces[uuid] = traces

## Get sz onset velocity for all recordings

In [ ]:
WIN_SIZE = 150  # take 10 s window from start of sz
N_EXTRA_FRAMES_BEFORE = 0  # add extra time to be included before sz begin

N_PIXELS = 512
UM_PER_PIXEL = 1.579  # µm per pixel. divide by 1000 to convert to mm

In [ ]:
for uuid in dict_uuid_traces:
    traces = dict_uuid_traces[uuid]
    # get min and max around Sz time window
    df_segments = ddoc.getSegmentsForUUID(uuid)
    for i_sz, sz_row in df_segments[df_segments["interval_type"] == "sz"].iterrows():
        i_frame_begin = sz_row.frame_begin - N_EXTRA_FRAMES_BEFORE - 1  # bring to 0-indexing
        i_frame_end = i_frame_begin + WIN_SIZE
        # define start of onset as earliest time point where a neuron's signal reaches local maximum (within time window)
        # define end of onset as latest such time point
        # take a subset of cells with 5% outliers (deviation from mean max time) removed
        max_times = np.argmax(traces[:, i_frame_begin:i_frame_end], axis=1)  # the time points of each neuron where they reached max for the first time
        deviations = np.abs(max_times - np.mean(max_times))  # absolute difference from mean onset time
        i_deviations_minus_outliers = np.argsort(deviations)[:-int(0.05*len(max_times))]  # exclude top 5% absolute differences
        onsets_filtered = max_times[i_deviations_minus_outliers]
        earliest_onset = np.min(onsets_filtered)  # earliest reached maximum in cells without 5% outliers
        latest_onset = np.max(onsets_filtered)
        mean_onset = np.mean(onsets_filtered)

In [ ]:
df_segments = ddoc.getSegmentsForUUID("4fe45b25dc854453880cd868fe77e9d4")

In [ ]:
for i, row in df_segments[df_segments["interval_type"] == "sz"].iterrows():
    print(row.frame_begin)

## Plot example

In [ ]:
sample_uuid = list(dict_uuid_traces.keys())[0]
sample_traces = dict_uuid_traces[sample_uuid]

In [ ]:
df_segments = ddoc.getSegmentsForUUID(sample_uuid)

In [ ]:
n_sz = len(df_segments[df_segments["interval_type"] == "sz"])

In [ ]:
i_sz = 0

In [ ]:
sz_data = df_segments[df_segments["interval_type"] == "sz"].iloc[i_sz]
i_frame_begin = sz_data.frame_begin
i_frame_end = min(i_frame_begin + 300, sz_data.frame_end)
if i_frame_end - i_frame_begin < 300:
    print("Used sz_data.frame_end")

In [ ]:
n_frames_before = 30  # 2 s before seizure begin

In [ ]:
max_traces = np.max(traces[:, i_frame_begin-n_frames_before:i_frame_end], axis=1)
min_traces = np.min(traces[:, i_frame_begin-n_frames_before:i_frame_end], axis=1)


In [ ]:
max_traces_arr = np.repeat(max_traces, traces.shape[1]).reshape(traces.shape)
min_traces_arr = np.repeat(min_traces, traces.shape[1]).reshape(traces.shape)


In [ ]:
# normalize traces for plotting
traces_norm = (traces - min_traces_arr)/(max_traces_arr - min_traces_arr)

In [ ]:
onset_ends = np.argmax(traces_norm[:, i_frame_begin-n_frames_before:i_frame_begin+150], axis=1)

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.hist(onset_ends, bins=30)
plt.show()

In [ ]:
# TODO: throw away 5% outliers
deviations = np.abs(onset_ends - np.mean(onset_ends))  # absolute difference from mean
i_deviations_minus_outliers = np.argsort(deviations)[:-int(0.05*len(onset_ends))]  # exclude top 5% absolute differences
onsets_filtered = onset_ends[i_deviations_minus_outliers]
lower_thr = np.min(onsets_filtered)  # earliest reached maximum in cells without 5% outliers
upper_thr = np.max(onsets_filtered)
mean_onset = np.mean(onsets_filtered)

In [ ]:
i_begin_onset = i_frame_begin - n_frames_before + lower_thr  # change this
i_end_onset = i_frame_begin - n_frames_before + upper_thr#int(i_frame_begin-n_frames_before + mean_onset)  # change this
if i_sz < n_sz:
    fig = plt.figure(figsize=(18,18))
    for trace in traces_norm:
        plt.plot(trace)
    plt.vlines(x=[i_begin_onset, i_end_onset], ymin=0, ymax=1.02, color="red")
    plt.xlim((i_frame_begin-20, i_frame_end))
    plt.ylim((0, 1.05))
    plt.show()
    print(f"Begin: {i_begin_onset}, end: {i_end_onset}")
else:
    print(f"No sz with index {i_sz} detected")

In [ ]:


# estimate velocity by "distance" (one dimension of FOV in mm) divided by onset time in min (dt in frames given, assuming 15 Hz )
v_onset_mmpmin = (N_PIXELS*UM_PER_PIXEL/1000.)/((i_end_onset - i_begin_onset)/(15*60))

In [ ]:
v_onset_mmpmin